In [44]:
import requests
import time
import re 
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from collections import Counter
import datetime

pd.set_option("display.max_colwidth", 80)

In [15]:
baseurl = "https://baseconnect.in"
category_url = "/companies/category/63b80d21-f5eb-43d1-8c1c-ed1bf3a15cb5"
testurl = baseurl + category_url

# user agent
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
}

In [4]:
res = requests.get(baseurl, headers=headers)
soup = BeautifulSoup(res.content, "html.parser")
# tests = soup.find_all("div", "searches__result__list")

In [210]:
address= [test.find("a").get("href") for test in tests]
address

['/companies/c1c16608-d72b-49e6-b825-7ce1ca242b1c',
 '/companies/a8f6b6e5-30e0-4bf8-9fd5-a2aa325dc6e5',
 '/companies/4f477b52-b939-43b5-89fc-66803bada4a0',
 '/companies/1f5dfbec-c79f-40a3-ad01-29f9b1b47fa3',
 '/companies/79230993-4306-47fd-a340-d2384ff5b0af',
 '/companies/a8405157-1833-43ea-ada5-3f59c824d79e',
 '/companies/562bb360-f274-4ffa-ad20-465d95e5b356',
 '/companies/af558909-82e8-4fe7-a2c7-09997603f83e',
 '/companies/66a4ddba-3daa-4d31-99f0-437a656fcd3c',
 '/companies/91014f17-3dda-4dd5-b6c8-09b5d5c9752f',
 '/companies/d834439e-6164-4e9c-8223-8da59f7d7f8a',
 '/companies/c950c6c0-7a61-44ad-bbb0-d16c34e06646',
 '/companies/e40a4d13-14c2-450c-bccf-f3ee078c7851',
 '/companies/3876f1da-d2c3-40f9-92ca-75e380b46922',
 '/companies/a9713259-4135-4428-97ca-700034068328',
 '/companies/83c678cc-bb96-4cdf-90a7-f7a3e0a68929',
 '/companies/ebf84a84-26ec-40b5-bcd8-8457029d4acf',
 '/companies/617ba75b-9967-4283-bb1c-016a554bceed',
 '/companies/f3b1b241-3f21-420b-99f2-da7cb10bedfb',
 '/companies

In [52]:
os.path.join(datetime.datetime.now().strftime('%Y%m%d%H%M'), "backup")

'201808241311/backup'

In [42]:
# カテゴリの取得
def remove_space(str_in):
    return re.sub(r"\s+", "", str_in)

category_dict = {}
keyword_dict = {}
keyword = "建設, 不動産, IT"

res = requests.get("http://baseconnect.in/", headers=headers)
soup = BeautifulSoup(res.content, "html.parser")

for item in soup.find("div", "home__category--left").find_all("li"):
    category_dict[re.sub(r"・$", "", item.p.string)] = baseurl+item.a.get("href")

for item in soup.find("div", "home__category--right").find_all("li"):
    category_dict[re.sub(r"・$", "", item.p.string)] = baseurl+item.a.get("href")
    
# 医療系のリストの作成
if not keyword == "":
    keyword_list =list(map(remove_space, keyword.split(",")))
    for keyword_ in keyword_list:
        for genre in soup.find_all(class_="node__box node__box--home home__category__box"):
            if keyword_ in genre.h3.string:
                for item in genre.find_all("li"):
                    keyword_dict[re.sub(r"・$", "", item.p.string)] = baseurl+item.a.get("href")
        category_dict = keyword_dict
    
category_dict

{'Webサービス・アプリ運営業界の会社': 'https://baseconnect.in/companies/category/b1d5c1e6-7cc5-41c4-9552-28530d2c9e9c',
 'eコマース業界の会社': 'https://baseconnect.in/companies/category/31fbf965-76bc-40ec-ac9f-37060983fbae',
 'その他不動産業界': 'https://baseconnect.in/companies/category/dbd1c236-96a0-4cea-b53e-5616d5add040',
 'その他土木工事業界の会社': 'https://baseconnect.in/companies/category/c3a7d969-d95d-46c2-9556-71a8785c860d',
 'その他建設・工事業界の会社': 'https://baseconnect.in/companies/category/889ba720-a85e-4039-9401-51590bdd42a2',
 'その他電気設備工事業界の会社': 'https://baseconnect.in/companies/category/4981132b-92cd-4900-970d-8ac0ed7ee6cf',
 'インテリアデザイン業界の会社': 'https://baseconnect.in/companies/category/0cdb9e58-df07-4ccc-8f83-b315f70db10a',
 'クラウド・フィンテック業界の会社': 'https://baseconnect.in/companies/category/578d6793-48f8-4776-906f-756a0b42f195',
 'システム受託開発業界の会社': 'https://baseconnect.in/companies/category/a2b5e24f-c628-44e5-a341-b6e4a6e4f0e3',
 'システム開発業界の会社': 'https://baseconnect.in/companies/category/377d61f9-f6d3-4474-a6aa-4f14e3fd9b17',
 

In [32]:
# 初期化
# user agent
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
}

target_category = list(category_dict.keys())[index_start:index_end]

#### 業界の選択 ####
for key, category_url in category_dict.items():
    if not key in target_category: # test
        continue
    print(key)
    # 初期化
    page = 1
    last_flag = False
    df_all = pd.DataFrame()

    while True:
        url = category_url + "/" + str("?page=") + str(page)

        # ページの読み込み
        time.sleep(3) # 遅延させる
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, "html.parser")
        
        # 企業ページリストの取得
        lists = soup.find_all("div", "searches__result__list")
        address= [test.find("a").get("href") for test in lists]
        if len(address) == 0:
            print("no list")
            break

        #### 業界のリスト ####
        if soup.find(class_="next_page disabled") is None or last_flag is True:
            # ページ表示
            print("--- Getting data at page = {} ---".format(page))

            for target in tqdm(address):
                # print(target)
                # time.sleep(1)
                res = requests.get(baseurl+target, headers=headers)
                soup = BeautifulSoup(res.content, "html.parser")

                # 会社名の抽出
                basic_origin = {}
                
                if soup.find("h1", "node__header__text__title") is None:
                    continue
                company_name = soup.find("h1", "node__header__text__title").get_text().strip()
                company_kana = soup.find("div", "node__header__title__reading").get_text().strip()

                basic_origin["会社名"] = company_name
                basic_origin["会社名_ふりがな"] = company_kana

                # 説明の抽出
                company_title = soup.find("div", "node__header__cont__text--thumb").find("h2").get_text().strip()
                company_abst = soup.find("div", "node__header__cont__text--thumb").find("p").get_text().strip()

                basic_origin["会社概要_タイトル"] = company_title
                basic_origin["会社概要_内容"] = company_abst

                # タグの抽出
                company_tag = soup.find("ul", "node__header__text__title__tag").get_text().strip().replace("\n", " ")
                basic_origin["会社_タグ"] = company_tag

                # 特徴と事業内容キーワード
                company_keyword_character = soup.find("div", "node__header__tag__wrapper")
                for content in company_keyword_character.find_all(class_="node__header__tag"):
                    if "事業内容" in content.find("h3").get_text():
                        company_key = content.find("ul").get_text().strip().replace("\n", " ")
                        basic_origin["会社_事業内容キーワード"] = company_key
                    if "特徴" in content.find("h3").get_text():
                        company_char = content.find("ul").get_text().strip().replace("\n", " ")
                        basic_origin["会社_特徴"] = company_char

                basic_info2 = {}
                rows = soup.find_all("dl")
                for row in rows:
                    basic_info2[re.sub(r"\s+", "", row.dt.get_text())] = re.sub(r"\s+", "", row.dd.get_text())

                try:
                    basic_info2["代表者_名前"] = basic_info2.pop("名前")
                except KeyError:
                    pass
                try:
                    basic_info2["代表者_出身大学"] = basic_info2.pop("出身大学")
                except KeyError:
                    pass
                try:
                    basic_info2["代表者_生年月日"] = basic_info2.pop("生年月日")
                except KeyError:
                    pass


                # 代表者
                basic_officer_all = {}
                soup_tmp = soup.find("div", "node__officer")
                if soup_tmp:
                    rows = soup_tmp.find_all(class_="nodeTable--desc")
                    for index, row in enumerate(rows):
                        basic_officer = {}
                        tmp = re.sub(r"\s+", " ", row.find("h3").get_text().strip().replace("\n", "")).split(" ")
                        basic_officer["氏名"], basic_officer["役職"] = tmp[0], ''.join(tmp[1:])
                        basic_officer["プロフィール"] = re.sub(r"\s+", "", row.find(class_="nodeTable--desc__desc").get_text().strip())    
                        basic_officer_all["役員_"+str(index).zfill(2)] = basic_officer


                # 連絡先
                basic_contact = {}
                soup_tmp = soup.find("div", "node__contact")
                if soup_tmp:
                    for item in soup_tmp.find_all("a"):
                        tmp = re.sub(r"\s+", "", item.get_text())
                        if "会社" in tmp:
                            basic_contact[tmp] = item.get("href")
                        elif "お問い合わせ" in tmp:
                            basic_contact[tmp] = item.get("href")


                # ランキング
                basic_rank = {}
                soup_tmp = soup.find("div", "nodeRank")
                if soup_tmp:
                    title = soup_tmp.find(class_="nodeRank__heading").string
                    for list_item in soup_tmp.find_all("li"):
                        tmp = re.sub(r"\s+", "", list_item.get_text())
                        rank, genre = re.search(r"(.*)(\(.*?\))", tmp).groups()
                        basic_rank[title+genre] = rank

                soup_tmp = soup.find("div", "nodeRank--other")
                if soup_tmp:
                    title = soup_tmp.find(class_="nodeRank__heading").string
                    for list_item in soup_tmp.find_all("li"):
                        tmp = re.sub(r"\s+", "", list_item.get_text())
                        rank, genre = re.search(r"(.*)(\(.*?\))", tmp).groups()
                        basic_rank[title+genre] = rank



                basic_origin.update(basic_info2)
                basic_origin.update(basic_officer_all)
                basic_origin.update(basic_contact)
                basic_origin.update(basic_rank)

                # データフレームに格納
                df = pd.io.json.json_normalize(basic_origin)
                try:
                    df_all = pd.concat([df_all, df])
                except NameError:
                    df_all = pd.io.json.json_normalize(basic_origin)
            # ページの追加
            page += 1

            if last_flag:
                df_all.to_csv("./"+key+".csv")
                print("--- DONE ---")
                break
        else:
            if last_flag is True:
                break
            else:
                last_flag = True

紙類包装資材業界の会社


  0%|          | 0/50 [00:00<?, ?it/s]

--- Getting data at page = 1 ---


 40%|████      | 20/50 [00:06<00:08,  3.48it/s]


KeyboardInterrupt: 

In [261]:
df_all

,FAX番号,お問い合わせページ,上場市場,事業所数,代表メールアドレス,代表者_出身大学,代表者_名前,代表者_生年月日,代表者電話番号,会社_タグ,...,売上順位(小業界:電線・ケーブル業界の会社),売上順位(本社都道府県:埼玉県),売上順位(本社都道府県:東京都),売上高,工場数,店舗数,採用メールアドレス,決算月,設立年月,資本金
0,情報あり,NaN,未上場,0~3箇所,NaN,NaN,平野豪,NaN,情報あり,電線・ケーブル業界の会社,...,NaN,NaN,NaN,NaN,0~5箇所,NaN,NaN,NaN,1976年6月,3000万円
0,情報あり,NaN,未上場,0~3箇所,情報あり,NaN,内山喜晴,NaN,情報あり,電線・ケーブル業界の会社 照明器具業界の会社 プラスチック成形機械業界の会社 LED業界の会社,...,NaN,NaN,NaN,NaN,NaN,0~3箇所,NaN,NaN,2011年3月,1000万円
0,情報あり,NaN,未上場,0~3箇所,NaN,NaN,別處耕治,NaN,情報あり,電線・ケーブル業界の会社,...,NaN,NaN,NaN,NaN,0~5箇所,NaN,NaN,NaN,1930年5月,1000万円
0,情報あり,http://www.kk-goko.co.jp/contact.htm,未上場,0~3箇所,NaN,NaN,瀧井茂,NaN,情報あり,電線・ケーブル業界の会社,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1968年6月,1000万円
0,情報あり,NaN,未上場,0~3箇所,情報あり,NaN,荒井貴,NaN,情報あり,プラスチック製品業界の会社 電線・ケーブル業界の会社 電子部品業界の会社,...,NaN,NaN,NaN,NaN,0~5箇所,NaN,NaN,情報あり,1973年2月,2000万円
0,情報あり,NaN,未上場,0~3箇所,NaN,NaN,及川雅晴,NaN,情報あり,電線・ケーブル業界の会社 溶接機械業界の会社,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,情報あり,NaN,未上場,0~3箇所,情報あり,NaN,藤田昇,NaN,情報あり,電線・ケーブル業界の会社 その他専門商社業界の会社,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015年10月,NaN
0,情報あり,http://www.cosmowave.jp/toi1.html,未上場,0~3箇所,NaN,NaN,荒井義浩,NaN,情報あり,電話機業界の会社 eコマース業界の会社 電線・ケーブル業界の会社,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006年8月,200万円
0,情報あり,NaN,未上場,0~3箇所,情報あり,NaN,加藤拓也,NaN,情報あり,電線・ケーブル業界の会社 窯業系建材業界の会社,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1947年3月,3600万円
0,情報あり,http://corpkimura.com/contact/index.html,未上場,0~3箇所,NaN,NaN,木村良行,NaN,情報あり,自動車部品・カー用品業界の会社 電線・ケーブル業界の会社,...,NaN,NaN,NaN,NaN,0~5箇所,NaN,NaN,NaN,NaN,3000万円


In [173]:
df_all.to_csv("out/base_test.csv")

In [35]:
tmp = pd.read_csv("./北海道.csv")

# データの結合

In [2]:
import os
target_dir = "/Users/YukiOta/Desktop/baseconnect/医療/"
filelist = os.listdir(target_dir)

In [3]:
df_all = pd.DataFrame()
for file_ in filelist:
    if file_.startswith("."):
        continue
    print(file_)
    tmp = pd.read_csv(os.path.join(target_dir, file_), low_memory=False)
    df_all = pd.concat([df_all, tmp])

高齢者向け福祉業界の会社.csv
病院業界の会社.csv
調査・検査・研究業界の会社.csv
高齢者住宅業界の会社.csv
医療機器・器具メーカー業界の会社.csv
障害者福祉業界の会社.csv
製薬業界の会社.csv
児童福祉業界の会社.csv
調剤薬局業界の会社.csv
介護業界の会社.csv
家庭用医療機器・介護用品業界の会社.csv


In [4]:
df_comp = df_all.drop_duplicates()

In [12]:
df_comp.to_csv("./out/data_baseconnect_医療_utf8.csv", encoding="utf-8")

In [58]:
df_comp

,FAX番号,Unnamed: 0,お問い合わせページ,上場市場,上場日,事業所数,代表メールアドレス,代表者_出身大学,代表者_名前,代表者_生年月日,...,従業員数順位(本社都道府県:青森県),従業員数順位(本社都道府県:静岡県),従業員数順位(本社都道府県:香川県),従業員数順位(本社都道府県:高知県),従業員数順位(本社都道府県:鹿児島県),採用メールアドレス,新卒採用人数,決算月,設立年月,資本金
0,情報あり,0,https://www.hitowa.com/holdings/contact/,未上場,NaN,0~3箇所,NaN,情報あり,今田健治,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11~50人,情報あり,2016年7月,1億円
1,情報あり,0,http://www.mcsg.co.jp/contact/,未上場,NaN,0~3箇所,NaN,情報あり,山本教雄,NaN,...,NaN,NaN,NaN,NaN,NaN,情報あり,11~50人,情報あり,1999年11月,1億円
2,情報あり,0,https://fofa.jp/ccfmp/a.p/101/,未上場,NaN,11箇所以上,NaN,情報あり,五郎丸徹,情報あり,...,NaN,NaN,NaN,NaN,NaN,情報あり,NaN,情報あり,2004年7月,9000万円
3,情報あり,0,https://www.yacs.jp/home/inquiry/form.html,未上場,NaN,0~3箇所,NaN,NaN,神﨑彰道,NaN,...,NaN,NaN,NaN,NaN,NaN,情報あり,51~100人,NaN,1960年9月,4億9100万円
4,情報あり,0,NaN,未上場,NaN,11箇所以上,NaN,NaN,吉田伸一,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,情報あり,1995年11月,7500万円
5,情報あり,0,http://www.grand-inc.co.jp/contact/index.html,未上場,NaN,11箇所以上,情報あり,NaN,福島裕至,NaN,...,NaN,NaN,NaN,NaN,NaN,情報あり,NaN,情報あり,1999年9月,1000万円
6,情報あり,0,http://dai-ichi.com/contact.html,未上場,NaN,4~10箇所,情報あり,情報あり,柴田義春,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961年5月,1000万円
7,情報あり,0,https://ginzam.com/contact/,未上場,NaN,11箇所以上,情報あり,NaN,高倉鉄夫,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,情報あり,2012年3月,1000万円
8,情報あり,0,https://ssl42.dsbsv.net/mws-hidaka.jp/publics/index/5/?ds=9518c7cb5c6f7fe3e6...,未上場,NaN,11箇所以上,NaN,情報あり,北嶋史誉,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,情報あり,1977年10月,300万円
9,情報あり,0,NaN,未上場,NaN,0~3箇所,情報あり,NaN,長南登志,NaN,...,NaN,NaN,NaN,NaN,NaN,情報あり,11~50人,情報あり,1989年11月,1000万円
